In [44]:
import plotly.graph_objects as go
import plotly.io as pio
import argparse
import pandas as pd

pio.renderers.default = 'browser'

ohlc_df = pd.read_csv(f'../data/btc.csv').tail(1000)
fig = go.Figure(data=[
    go.Candlestick(
        x=ohlc_df['Timestamp'], 
        open=ohlc_df['Open'], 
        high=ohlc_df['High'], 
        low=ohlc_df['Low'], 
        close=ohlc_df['Close'],
        increasing_line_color= '#29d691',
        decreasing_line_color= '#ee113e'
    )
])

positions_df = pd.read_csv(f'../results/closed_positions.csv').tail(1000)
first_ohlc_timestamp = ohlc_df['Timestamp'].iloc[0]
position_id = 1

for index, row in positions_df.iterrows():
    if row['Entry'] < first_ohlc_timestamp:
        continue

    color = 'green' if int(row['PnL']) > 0 else 'red'
    entry_low = ohlc_df.loc[ohlc_df['Timestamp'] == row['Entry'], 'Low'].values[0]
    exit_low = ohlc_df.loc[ohlc_df['Timestamp'] == row['Exit'], 'Low'].values[0]

    fig.add_trace(go.Scatter(
        x=[row['Entry']],
        y=[entry_low],
        mode='markers',
        marker=dict(
            size=20, 
            color='rgba(200, 34, 221, 0.4)', 
            symbol='circle'
        ),
        name='Entry',
        text=[f"Entry: {row['Entry']}<br>Position ID: {position_id}<br>Type: {row['Type'].capitalize()}"],
        hoverinfo='text'
    ))

    fig.add_trace(go.Scatter(
        x=[row['Exit']],
        y=[exit_low],
        mode='markers',
        marker=dict(
            size=20,
            symbol='circle',
            color=['rgba(0, 200, 0, 0.4)' if row['PnL'] > 0 else 'rgba(255, 0, 0, 0.4)']
        ),
        name='Exit',
        text=[f"Exit: {row['Exit']}<br>Position ID: {position_id}<br>Type: {row['Type'].capitalize()}<br>PnL: {row['PnL']}"],
        hoverinfo='text'
    ))

    position_id += 1

fig.show()
